In [3]:
import pandas as pd
import pyodbc
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [4]:
import mysql.connector

In [7]:
conn = mysql.connector.connect(
    host = 'localhost',
    user = 'root',
    password = 'C5V7A9X8!c',
    database = 'campaignanalysis'
)
cursor = conn.cursor()

In [8]:
query = 'SELECT * FROM `dbo.customer_reviews`;'
df = pd.read_sql(query,conn)

C:\Users\shars\AppData\Local\Temp\ipykernel_17060\3012554261.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query,conn)


In [9]:
df

,ReviewID,CustomerID,ProductID,ReviewDate,Rating,ReviewText
0,1,77,18,2023-12-23,3,"Average experience, nothing special."
1,2,80,19,2024-12-25,5,The quality is top-notch.
2,3,50,13,2025-01-26,4,Five stars for the quick delivery.
3,4,78,15,2025-04-21,3,"Good quality, but could be cheaper."
4,5,64,2,2023-07-16,3,"Average experience, nothing special."
...,...,...,...,...,...,...
1358,1359,28,4,2023-05-25,3,Not worth the money.
1359,1360,58,12,2023-11-13,2,"Average experience, nothing special."
1360,1361,96,15,2023-03-07,5,Customer support was very helpful.
1361,1362,99,2,2025-12-03,1,Product did not meet my expectations.


In [12]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\shars\AppData\Roaming\nltk_data...


True

In [13]:
sia = SentimentIntensityAnalyzer()

In [14]:
def calculate_review(review):
    sentiment = sia.polarity_scores(review)
    return sentiment['compound']

In [15]:
def categorize_sentiment(score,rating):
    if score>0.05:
        if rating>=4:
            return 'Positive'
        elif rating == 3:
            return 'Mixed Positive'
        else:
            return 'Mixed Neutral'
    elif score<-0.05:
        if rating<=2:
            return 'Negative'
        elif rating == 3:
            return 'Mixed Negative'
        else:
            return 'Mixed Neutral'
    else:
        if rating>=4:
            return 'Positive'
        elif rating<=2:
            return 'Negative'
        else:
            return 'Neutral'

In [17]:
def sentiment_bucket(score):
    if score>=0.5:
        return '0.5 to 1'
    elif 0.0<= score <0.5:
        return '0.0 to 0.49'
    elif -0.5<=score<0.0:
        return '-.0.5 to -0.0'
    else:
        return '-1.0 to -0.49'

In [18]:
df['Sentimentscore'] = df['ReviewText'].apply(calculate_review)

In [19]:
df['Sentimentcategory'] = df.apply(lambda row : categorize_sentiment(row['Sentimentscore'],row['Rating']),axis=1)

In [20]:
df['Sentimentbucket'] = df['Sentimentscore'].apply(sentiment_bucket)

In [21]:
df

,ReviewID,CustomerID,ProductID,ReviewDate,Rating,ReviewText,Sentimentscore,Sentimentcategory,Sentimentbucket
0,1,77,18,2023-12-23,3,"Average experience, nothing special.",-0.3089,Mixed Negative,-.0.5 to -0.0
1,2,80,19,2024-12-25,5,The quality is top-notch.,0.0000,Postive,0.0 tp 0.49
2,3,50,13,2025-01-26,4,Five stars for the quick delivery.,0.0000,Postive,0.0 tp 0.49
3,4,78,15,2025-04-21,3,"Good quality, but could be cheaper.",0.2382,Mixed Positive,0.0 tp 0.49
4,5,64,2,2023-07-16,3,"Average experience, nothing special.",-0.3089,Mixed Negative,-.0.5 to -0.0
...,...,...,...,...,...,...,...,...,...
1358,1359,28,4,2023-05-25,3,Not worth the money.,-0.1695,Mixed Negative,-.0.5 to -0.0
1359,1360,58,12,2023-11-13,2,"Average experience, nothing special.",-0.3089,Negative,-.0.5 to -0.0
1360,1361,96,15,2023-03-07,5,Customer support was very helpful.,0.6997,Positive,0.5 to 1
1361,1362,99,2,2025-12-03,1,Product did not meet my expectations.,0.0000,Negative,0.0 tp 0.49


In [23]:
df.to_csv('fact_customer_reviews.csv',index=False)